In [10]:
import numpy as np
# pandas
from pandas import read_csv
# matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
#teras
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC, Accuracy, Precision, Recall, FalsePositives
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix

In [2]:
seed = 13
np.random.seed(seed)


In [3]:
# load dataset
dataset = input('\nWhich is the name of the dataset? ')
dataframe = read_csv(dataset, header=0)
dataset = dataframe.values
#dataset = np.random.shuffle(dataset)
first_col = input("\nSelect first column: \n") #2
last_col = input('\nSelect last col: \n')          #15

X = dataset[:,int(first_col):int(last_col)].astype(float)   # columns from 3rd to 14th into X
Y = dataset[:,int(last_col)] #label column (15th) into Y 


Which is the name of the dataset? analysis.csv

Select first column: 
2

Select last col: 
15


In [24]:
#Encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

#One-hot encoding
transformed_Y = to_categorical(encoded_Y)

In [25]:
#Initialising NN
model = Sequential()

#First layer
model.add(Dense(8, activation='relu', input_shape=(13,)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

#Second layer
model.add(Dense(12, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(2, activation='sigmoid'))


model.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr= 0.0001),
              metrics=[Precision(name = 'precision'),
                       Recall(name = 'recall')]
             )

X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    transformed_Y,
                                                    test_size=0.2,
                                                    random_state=seed,
                                                    shuffle = True)


In [ ]:
epochs = 50
val_data = (X_test,Y_test)
checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_precision', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history=model.fit(X_train,
                  Y_train,
                  epochs=epochs,
                  validation_data=val_data,
                  batch_size=150000)

Train on 3040026 samples, validate on 760007 samples
Epoch 1/50
3040026/3040026 [==============================] - 2s 1us/sample - loss: 0.9211 - precision: 0.3842 - recall: 0.2218 - val_loss: 0.8262 - val_precision: 0.5640 - val_recall: 0.8896
Epoch 2/50
3040026/3040026 [==============================] - 2s 1us/sample - loss: 0.9089 - precision: 0.3845 - recall: 0.2222 - val_loss: 0.7753 - val_precision: 0.5761 - val_recall: 0.6458
Epoch 3/50
3040026/3040026 [==============================] - 2s 1us/sample - loss: 0.8978 - precision: 0.3857 - recall: 0.2230 - val_loss: 0.7617 - val_precision: 0.6367 - val_recall: 0.4363
Epoch 4/50
3040026/3040026 [==============================] - 2s 1us/sample - loss: 0.8865 - precision: 0.3870 - recall: 0.2241 - val_loss: 0.7575 - val_precision: 0.6046 - val_recall: 0.3078
Epoch 5/50
3040026/3040026 [==============================] - 2s 1us/sample - loss: 0.8756 - precision: 0.3893 - recall: 0.2253 - val_loss: 0.7566 - val_precision: 0.5807 - val_re

3040026/3040026 [==============================] - 2s 1us/sample - loss: 0.6317 - precision: 0.7453 - recall: 0.5338 - val_loss: 0.6034 - val_precision: 0.9339 - val_recall: 0.6076
Epoch 44/50
3040026/3040026 [==============================] - 2s 1us/sample - loss: 0.6267 - precision: 0.7553 - recall: 0.5487 - val_loss: 0.5990 - val_precision: 0.9393 - val_recall: 0.6455
Epoch 45/50
3040026/3040026 [==============================] - 2s 1us/sample - loss: 0.6224 - precision: 0.7637 - recall: 0.5595 - val_loss: 0.5947 - val_precision: 0.9440 - val_recall: 0.6825
Epoch 46/50
3040026/3040026 [==============================] - 2s 1us/sample - loss: 0.6171 - precision: 0.7750 - recall: 0.5748 - val_loss: 0.5904 - val_precision: 0.9483 - val_recall: 0.7206
Epoch 47/50
3040026/3040026 [==============================] - 2s 1us/sample - loss: 0.6128 - precision: 0.7838 - recall: 0.5890 - val_loss: 0.5861 - val_precision: 0.9520 - val_recall: 0.7560
Epoch 48/50
3000000/3040026 [==================

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# show the confusion matrix of our predictions
#compute predictions
predictions = model.predict(X_test)

y_pred = np.array([np.argmax(probas) for probas in predictions])
y_test = np.array([np.argmax(label) for label in Y_test])




In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
print (cm)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()  #Clears the figure
prec = history.history['precision']
val_prec = history.history['val_precision']
plt.plot(epochs, prec, 'bo', label='Training precision')
plt.plot(epochs, val_prec, 'r', label='Validation precision')
plt.title('Training and validation precision')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()  #Clears the figure
rec = history.history['recall']
val_rec = history.history['val_recall']
plt.plot(epochs, rec, 'bo', label='Training recall')
plt.plot(epochs, val_rec, 'r', label='Validation recall')
plt.title('Training and validation recall')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()